In [18]:
import pandas as pd
import os

from glob import glob

from sshtunnel import SSHTunnelForwarder

import paramiko
import geopandas as gpd
import numpy as np
import json
import rasterio
import sshtunnel
import MySQLdb

from sqlalchemy import (
    Connection,
    CursorResult,
    Engine,
    Select,
    TextClause,
    create_engine,
    func,
    select,
)

In [2]:
pkl_files = glob(os.path.join('./OSN_pickles', '**', '*.pkl'), recursive=True)

In [6]:
osn_data = pd.read_pickle(pkl_files[10])

In [7]:
osn_data.geoaltitude.unique()

array([266.7 , 259.08, 251.46, 243.84, 243.84, 236.22, 228.6 , 220.98,
       213.36, 205.74, 198.12, 190.5 , 190.5 , 182.88, 175.26, 167.64,
       160.02, 289.56, 281.94, 274.32, 266.7 , 259.08, 251.46, 274.32,
       297.18, 304.8 , 312.42, 320.04, 594.36, 601.98, 487.68, 495.3 ,
       480.06, 472.44, 441.96, 434.34, 426.72, 419.1 , 411.48, 403.86,
       396.24, 388.62, 381.  , 464.82, 457.2 , 472.44, 480.06, 449.58,
       358.14, 365.76, 373.38, 350.52, 342.9 , 335.28, 327.66, 464.82,
       373.38, 358.14, 365.76, 571.5 , 579.12, 563.88, 609.6 , 297.18,
       304.8 , 236.22, 228.6 , 220.98, 182.88, 175.26, 167.64, 152.4 ,
       312.42, 320.04, 327.66, 342.9 , 381.  , 388.62, 396.24, 746.76,
       678.18, 685.8 , 693.42, 701.04, 708.66, 716.28, 723.9 , 746.76,
       739.14, 731.52, 251.46, 434.34, 350.52, 335.28, 487.68, 495.3 ,
       502.92, 144.78, 137.16, 129.54, 121.92, 510.54, 510.54, 518.16,
       670.56, 662.94, 655.32, 647.7 , 640.08, 632.46, 624.84, 617.22,
      

In [9]:
gdf_files = glob(os.path.join('./dataframes', '*', 'gdf*.json'))  # First level
gdf_files.extend(glob(os.path.join('./dataframes', 'gdf*.json')))  # Base directory


In [27]:
gdf_data = pd.DataFrame()

df_list = []

for file in gdf_files:

    temp_data = pd.read_json(file)

    df_list.append(temp_data)

gdf_data = pd.concat(df_list)
gdf_data.reset_index()

,index,time,icao24,lat,lon,velocity,heading,callsign,onground,squawk,...,hour,ref,inf_flt,inf_pt,dist,gnd_elev,gnd_inf_flt,gnd_inf_pt,min_hgt,lidar_min_alt
0,0,1659957628,3dd3bf,50.896539,6.991425,40.222152,339.015102,DHALB,false,7043,...,1659956400,3dd3bf_1_080822,True,False,0.000000,48.438858,True,True,348.438858,NaN
1,1,1659957629,3dd3bf,50.896729,6.991313,40.702893,339.274441,DHALB,false,7043,...,1659956400,3dd3bf_1_080822,True,False,22.452396,42.842587,True,True,342.842587,NaN
2,2,1659957630,3dd3bf,50.897141,6.991016,40.523691,339.954751,DHALB,false,7043,...,1659956400,3dd3bf_1_080822,True,False,72.774262,38.378052,True,True,338.378052,NaN
3,3,1659957631,3dd3bf,50.897470,6.990891,41.007351,340.201124,DHALB,false,7043,...,1659956400,3dd3bf_1_080822,True,False,110.486456,37.173000,True,False,337.173000,NaN
4,4,1659957632,3dd3bf,50.897827,6.990645,41.007351,340.201124,DHALB,false,7043,...,1659956400,3dd3bf_1_080822,True,False,153.742445,37.172001,True,False,337.172001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380479,2150,1698157708,3ff8d0,50.897186,6.921906,40.510628,310.364537,DMFJS,False,7000,...,1698156000,3ff8d0_1_241023,False,False,127.968517,63.442432,False,False,363.442432,395.51
380480,2151,1698157709,3ff8d0,50.897424,6.921463,40.510628,310.364537,DMFJS,False,7000,...,1698156000,3ff8d0_1_241023,False,False,168.756031,63.539776,False,False,363.539776,395.51
380481,2152,1698157710,3ff8d0,50.897598,6.921090,40.903960,309.897835,DMFJS,False,7000,...,1698156000,3ff8d0_1_241023,False,False,201.343082,62.125084,False,False,362.125084,395.51
380482,2153,1698157711,3ff8d0,50.897873,6.920644,41.299954,309.440053,DMFJS,False,7000,...,1698156000,3ff8d0_1_241023,False,False,245.028448,59.539700,False,False,359.539700,395.51


In [13]:
gdf_data.columns

Index(['time', 'icao24', 'lat', 'lon', 'velocity', 'heading', 'callsign',
       'onground', 'squawk', 'baroaltitude', 'geoaltitude', 'lastposupdate',
       'lastcontact', 'hour', 'ref', 'inf_flt', 'inf_pt', 'dist', 'gnd_elev',
       'gnd_inf_flt', 'gnd_inf_pt', 'min_hgt', 'lidar_min_alt'],
      dtype='object')

In [30]:
gdf_data['positionnac'] = 10
gdf_data['version'] = 2



In [31]:

MYSQL_secrets_json = './mysql_secrets.json'


with open(MYSQL_secrets_json) as MYSQL_secrets:
  MYSQL_creds = json.load(MYSQL_secrets)


In [32]:
gdf_data = gdf_data.drop(columns=['hour', 'lidar_min_alt'])

In [33]:
ed25519_key = paramiko.Ed25519Key(filename='/home/dimitri/.ssh/id_ed25519')

# Set up the SSH tunnel with the RSA key
with SSHTunnelForwarder(
    (MYSQL_creds['SSH_ADDRESS'], 22),
    ssh_username=MYSQL_creds['SSH_USERNAME'],
    ssh_pkey=ed25519_key,  # Use the loaded RSA key
    remote_bind_address=(MYSQL_creds['REMOTE_BIND_ADDRESS'], MYSQL_creds['REMOTE_BIND_PORT']),
    allow_agent=False,
) as tunnel:
    
    print('connected')

    engstr =  'mysql+pymysql://' + MYSQL_creds['SSH_USERNAME'] + ':' + MYSQL_creds['PYANYWHERE_PASSWORD'] + '@127.0.0.1:' + str(tunnel.local_bind_port)+'/dme3$' + MYSQL_creds['TEST_DATABASE_NAME']

    engine = create_engine(engstr)

    print('step 1')
    
    gdf_data.to_sql(con=engine, name = MYSQL_creds['MAIN_TEST_TABLE_NAME'], if_exists='append')

    print("Done")

connected
step 1
Done


# Infraction tables

In [40]:
inf_files = glob(os.path.join('./dataframes', '*', 'clean_inf_*.json'))  # First level
inf_files.extend(glob(os.path.join('./dataframes', 'clean_inf*.json')))  # Base directory
gnd_inf_files = glob(os.path.join('./dataframes', '*', 'clean_gnd_inf_*.json'))  # First level
gnd_inf_files.extend(glob(os.path.join('./dataframes', 'clean_gnd_inf*.json')))  # Base directory

In [41]:
inf_data = pd.DataFrame()
gnd_inf_data = pd.DataFrame()

inf_list = []
gnd_inf_list = []

for file in inf_files:

    temp_data = pd.read_json(file)

    inf_list.append(temp_data)

inf_data = pd.concat(inf_list)
inf_data.reset_index()

for file in gnd_inf_files:

    temp_data = pd.read_json(file)

    gnd_inf_list.append(temp_data)

gnd_inf_data = pd.concat(gnd_inf_list)
gnd_inf_data.reset_index()

inf_data.rename(columns={'timestamp_cpa': 'time'}, inplace=True)

gnd_inf_data.rename(columns={'timestamp_gprox': 'time'}, inplace=True)

gnd_inf_data['obstacle_name'] = 'ground'


In [43]:
inf_data = inf_data.drop(columns=['hour'])
gnd_inf_data = gnd_inf_data.drop(columns=['hour'])

In [44]:
ed25519_key = paramiko.Ed25519Key(filename='/home/dimitri/.ssh/id_ed25519')

# Set up the SSH tunnel with the RSA key
with SSHTunnelForwarder(
    (MYSQL_creds['SSH_ADDRESS'], 22),
    ssh_username=MYSQL_creds['SSH_USERNAME'],
    ssh_pkey=ed25519_key,  # Use the loaded RSA key
    remote_bind_address=(MYSQL_creds['REMOTE_BIND_ADDRESS'], MYSQL_creds['REMOTE_BIND_PORT']),
    allow_agent=False,
) as tunnel:
    
    print('connected')

    engstr =  'mysql+pymysql://' + MYSQL_creds['SSH_USERNAME'] + ':' + MYSQL_creds['PYANYWHERE_PASSWORD'] + '@127.0.0.1:' + str(tunnel.local_bind_port)+'/dme3$' + MYSQL_creds['TEST_DATABASE_NAME']

    engine = create_engine(engstr)

    print('step 1')

    inf_data.to_sql(con=engine, name = MYSQL_creds['INF_TEST_TABLE_NAME'], if_exists='append')
    gnd_inf_data.to_sql(con=engine, name = MYSQL_creds['INF_TEST_TABLE_NAME'], if_exists='append')

    print("Done")

connected
step 1
Done
